
**Install requirements**

In [0]:
!pip3 install 'torch==1.3.1'
!pip3 install 'torchvision==0.5.0'
!pip3 install 'Pillow-SIMD'
!pip3 install 'tqdm'

     |████████████████████████████████| 734.6MB 24kB/s 
ERROR: torchvision 0.5.0 has requirement torch==1.4.0, but you'll have torch 1.3.1 which is incompatible.
  Found existing installation: torch 1.4.0
    Uninstalling torch-1.4.0:
      Successfully uninstalled torch-1.4.0
     |████████████████████████████████| 753.4MB 20kB/s 
  Found existing installation: torch 1.3.1
    Uninstalling torch-1.3.1:
      Successfully uninstalled torch-1.3.1
     |████████████████████████████████| 634kB 6.4MB/s 
  Created wheel for Pillow-SIMD: filename=Pillow_SIMD-7.0.0.post3-cp36-cp36m-linux_x86_64.whl size=1110246 sha256=83594108c424ce18ca2729219071ef088fc22f36e9c5977c7a7d9657d151305f
  Stored in directory: /root/.cache/pip/wheels/d3/ac/4f/4cdf8febba528e5f1b09fc58d5181e1c12ed1e8655dcd583b8
Successfully built Pillow-SIMD


**Import libraries**

In [1]:
import os
import logging

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

import torchvision
from torchvision import transforms
from torchvision.models import alexnet
from torchvision.models import resnet101
from torchvision.models import resnet18
from PIL import Image
from tqdm import tqdm

ModuleNotFoundError: No module named 'torch'

**Set Arguments**

In [0]:
DEVICE = 'cuda' # 'cuda' or 'cpu'

NUM_CLASSES = 101 # 101 + 1: There is am extra Background class that should be removed 

#BATCH_SIZE = 512   # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
BATCH_SIZE = 256  # the batch size, learning rate should change by the same factor to have comparable results

LR = 0.01           # The initial Learning Rate
#LR = 1e-3
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default

NUM_EPOCHS = 30      # Total number of training epochs (iterations over dataset)
#NUM_EPOCH = 30
STEP_SIZE = 20       # How many epochs before decreasing learning rate (if using a step-down policy)
#STEP_SIZE = 20
GAMMA = 0.1          # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10


**Define Data Preprocessing**

In [0]:
# Define transforms for training phase
train_transform = transforms.Compose([transforms.Resize(256),      # Resizes short size of the PIL image to 256
                                      transforms.CenterCrop(224),  # Crops a central square patch of the image
                                                                   # 224 because torchvision's AlexNet needs a 224x224 input!
                                                                   # Remember this when applying different transformations, otherwise you get an error
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalizes tensor with mean and standard deviation
])
# Define transforms for the evaluation phase
eval_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))                                    
])
#part 3
# transforms for pretrained net
imageNet_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
#part 4
#transforms for data augmentation
imageNet_transform_train = transforms.Compose([transforms.Resize(256),
                                      #transforms.RandomHorizontalFlip(),
                                      #transforms.RandomGrayscale(0.5),
                                      #transforms.RandomRotation(30),
                                      #transforms.ColorJitter(),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      #transforms.RandomErasing(),
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

**Caltech Class**

In [0]:
from torchvision.datasets import VisionDataset

from PIL import Image

import os
import os.path
import sys


def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')


class Caltech(VisionDataset):
    def __init__(self, root, split='train', transform=None, target_transform=None):
        super(Caltech, self).__init__(root, transform=transform, target_transform=target_transform)

        self.split = split # This defines the split you are going to use
                           # (split files are called 'train.txt' and 'test.txt')

        '''
        - Here you should implement the logic for reading the splits files and accessing elements
        - If the RAM size allows it, it is faster to store all data in memory
        - PyTorch Dataset classes use indexes to read elements
        - You should provide a way for the __getitem__ method to access the image-label pair
          through the index
        - Labels should start from 0, so for Caltech you will have lables 0...100 (excluding the background class) 
        '''
        file = open(self.split, "r")
        lines = file.readlines() #load the text file
        file.close()
        self.elements = [i for i in lines if not i.startswith("BACKGROUND_Google")] #filter background class 
        self.classes = os.listdir(self.root) #in the root there are the names of the classes
        self.classes.remove("BACKGROUND_Google") #filter background class

    def __getitem__(self, index):
        '''
        __getitem__ should access an element through its index
        Args:
            index (int): Index
        Returns:
            tuple: (sample, target) where target is class_index of the target class.
        '''
        image_name = self.elements[index].rstrip()
        image = pil_loader(os.path.join(self.root, image_name)) #find the image in the right folder
        classname = self.elements[index].rstrip().split('/')[0]
        label = self.classes.index(classname) #take the index of the class of the image
         # Provide a way to access image and label via index
                                   # Image should be a PIL Image
                                   # label can be int

        # Applies preprocessing when accessing the image
        if self.transform is not None:
             image = self.transform(image)

        return image, label

    def __len__(self):
        '''
        The __len__ method returns the length of the dataset
        It is mandatory, as this is used by several other components
        '''
        length = len(self.elements) # Provide a way to get the length (number of elements) of the dataset
        return length

    def getclasses(self):
        return self.classes

**Prepare Dataset**

In [5]:
# Clone github repository with data
if not os.path.isdir('./Homework2-Caltech101'):
  !git clone https://github.com/MachineLearning2020/Homework2-Caltech101.git
  #!mv 'Homework2-Caltech101' 'Caltech101'

DATA_DIR = 'Homework2-Caltech101/101_ObjectCategories'
TRAIN_DIR = 'Homework2-Caltech101/train.txt'
TEST_DIR = 'Homework2-Caltech101/test.txt'
#from Caltech101.caltech_dataset import Caltech

# Prepare Pytorch train/test Datasets
train_val_dataset = Caltech(DATA_DIR, split=TRAIN_DIR, transform=train_transform)
test_dataset = Caltech(DATA_DIR, split=TEST_DIR, transform=eval_transform)

train_indexes = [idx for idx in range(len(train_val_dataset)) if idx % 2]
val_indexes = [idx for idx in range(len(train_val_dataset)) if not idx % 2]

train_dataset = Subset(train_val_dataset, train_indexes)
val_dataset = Subset(train_val_dataset, val_indexes)

# Check dataset sizes
print('Train Dataset: {}'.format(len(train_dataset)))
print('Valid Dataset: {}'.format(len(val_dataset)))
print('Test Dataset: {}'.format(len(test_dataset)))

#part 3 and 4
img_train_val_dataset = Caltech(DATA_DIR, split=TRAIN_DIR, transform=imageNet_transform)
#img_train_val_dataset = Caltech(DATA_DIR, split=TRAIN_DIR, transform=imageNet_transform_train) #for data augmentation
img_test = Caltech(DATA_DIR, split=TEST_DIR, transform=imageNet_transform)
img_train = Subset(img_train_val_dataset, train_indexes)
img_val = Subset(img_train_val_dataset, val_indexes)

Train Dataset: 2892
Valid Dataset: 2892
Test Dataset: 2893


**Prepare Dataloaders**

In [0]:
# Dataloaders iterate over pytorch datasets and transparently provide useful functions (e.g. parallelization and shuffling)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4) 
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

#part 3
img_train_dataloader = DataLoader(img_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
img_val_dataloader = DataLoader(img_val, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
img_test_dataloader = DataLoader(img_test, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

**Prepare Network**

In [0]:
net = alexnet() # Loading AlexNet model
best_model = alexnet() 
net_trained = alexnet(pretrained=True)
# AlexNet has 1000 output neurons, corresponding to the 1000 ImageNet's classes
# We need 101 outputs for Caltech-101
net.classifier[6] = nn.Linear(4096, NUM_CLASSES) # nn.Linear in pytorch is a fully connected layer
                                                 # The convolutional layer is nn.Conv2d
best_model.classifier[6] = nn.Linear(4096, NUM_CLASSES)
net_trained.classifier[6] = nn.Linear(4096, NUM_CLASSES)
# We just changed the last layer of AlexNet with a new fully connected layer with 101 outputs
# It is strongly suggested to study torchvision.models.alexnet source code

**Prepare Training**

In [0]:
# Define loss function
criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

# Choose parameters to optimize
# To access a different set of parameters, you have to access submodules of AlexNet
# (nn.Module objects, like AlexNet, implement the Composite Pattern)
# e.g.: parameters of the fully connected layers: net.classifier.parameters()
# e.g.: parameters of the convolutional layers: look at alexnet's source code ;) 

parameters_to_optimize = net.parameters() # In this case we optimize over all the parameters of AlexNet
pretrained_parameters_to_optimize = net_trained.parameters()
#UNCOMMENT to freeze layers
#pretrained_parameters_to_optimize = net_trained.features.parameters()
#pretrained_parameters_to_optimize = net_trained.classifier.parameters()

# Define optimizer
# An optimizer updates the weights based on loss
# We use SGD with momentum
optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
pretrained_optimizer = optim.SGD(pretrained_parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

# Define scheduler
# A scheduler dynamically changes learning rate
# The most common schedule is the step(-down), which multiplies learning rate by gamma every STEP_SIZE epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)
pretrained_scheduler = optim.lr_scheduler.StepLR(pretrained_optimizer, step_size=STEP_SIZE, gamma=GAMMA)

**Training from scratch**

In [0]:
import copy
# By default, everything is loaded to cpu
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

cudnn.benchmark # Calling this optimizes runtime

current_step = 0
best_acc = 0

training_loss = []
validation_loss = []
validation_accuracy = []

# Start iterating over the epochs
for epoch in range(NUM_EPOCHS):
  print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_lr()))
  loss_avg = 0
  # Iterate over the dataset
  for i,(images, labels) in enumerate(train_dataloader):
    # Bring data over the device of choice
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    net.train() # Sets module in training mode

    # PyTorch, by default, accumulates gradients after each backward pass
    # We need to manually set the gradients to zero before starting a new iteration
    optimizer.zero_grad() # Zero-ing the gradients

    # Forward pass to the network
    outputs = net(images)

    # Compute loss based on output and ground truth
    loss = criterion(outputs, labels)
    loss_avg += loss.item()
    # Log loss
    if current_step % LOG_FREQUENCY == 0:
      print('Step {}, Loss {}'.format(current_step, loss.item()))

    # Compute gradients for each layer and update weights
    loss.backward()  # backward pass: computes gradients
    optimizer.step() # update weights based on accumulated gradients

    current_step += 1

  #evaluate model at each epoch and take the best one

  net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
  net.train(False) # Set Network to evaluation mode

  running_corrects = 0
    val_loss_avg = 0
  for j,(images, labels) in enumerate(val_dataloader):
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

      # Forward Pass
    outputs = net(images)

      # Get predictions
    _, preds = torch.max(outputs.data, 1)

      # Update Corrects
    running_corrects += torch.sum(preds == labels.data).data.item()
    val_loss = criterion(outputs,labels)
    val_loss_avg += val_loss.item()
    # Calculate Accuracy
  accuracy = running_corrects / float(len(val_dataset))
  validation_accuracy.append(accuracy)
  validation_loss.append(val_loss_avg/float(j))
  print('\nValidation Accuracy: {}'.format(accuracy))
  if accuracy > best_acc:
    best_acc = accuracy
    best_model = copy.deepcopy(net)

  # Step the scheduler
  scheduler.step() 
  training_loss.append(loss_avg/float(i))

In [ ]:
from matplotlib import pyplot as plt

fig,ax = plt.subplots(2)
ax[0].plot(validation_accuracy, label = 'Validation Accuracy')
ax[1].plot(training_loss,label='Training Loss')
ax[1].plot(validation_loss,label = 'Validation Loss')
ax[0].grid()
ax[1].grid()
ax[0].legend()
ax[1].legend()

In [0]:
#test phase
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
net.train(False) # Set Network to evaluation mode

running_corrects = 0
for images, labels in tqdm(test_dataloader):
  images = images.to(DEVICE)
  labels = labels.to(DEVICE)

  # Forward Pass
  outputs = net(images)

  # Get predictions
  _, preds = torch.max(outputs.data, 1)

  # Update Corrects
  running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(test_dataset))

print('Test Accuracy: {}'.format(accuracy))

**Transfer Learning**

In [0]:
import copy
# By default, everything is loaded to cpu
net = net_trained.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

cudnn.benchmark # Calling this optimizes runtime

current_step = 0
best_acc = 0

training_loss = []
validation_loss = []
validation_accuracy = []

# Start iterating over the epochs
for epoch in range(NUM_EPOCHS):
  print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, pretrained_scheduler.get_lr()))
  loss_avg = 0
  # Iterate over the dataset
  for i,(images, labels) in enumerate(img_train_dataloader):
    # Bring data over the device of choice
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    net.train() # Sets module in training mode

    # PyTorch, by default, accumulates gradients after each backward pass
    # We need to manually set the gradients to zero before starting a new iteration
    pretrained_optimizer.zero_grad() # Zero-ing the gradients

    # Forward pass to the network
    outputs = net(images)

    # Compute loss based on output and ground truth
    loss = criterion(outputs, labels)
    loss_avg += loss.item()
    # Log loss
    if current_step % LOG_FREQUENCY == 0:
      print('Step {}, Loss {}'.format(current_step, loss.item()))

    # Compute gradients for each layer and update weights
    loss.backward()  # backward pass: computes gradients
    pretrained_optimizer.step() # update weights based on accumulated gradients

    current_step += 1

  #evaluate model at each epoch and take the best one

  net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
  net.train(False) # Set Network to evaluation mode

  running_corrects = 0
  val_loss_avg = 0
  for j,(images, labels) in enumerate(img_val_dataloader):
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

      # Forward Pass
    outputs = net(images)

      # Get predictions
    _, preds = torch.max(outputs.data, 1)

      # Update Corrects
    running_corrects += torch.sum(preds == labels.data).data.item()
    val_loss = criterion(outputs,labels)
    val_loss_avg += val_loss.item()
    # Calculate Accuracy
  accuracy = running_corrects / float(len(img_val))
  validation_accuracy.append(accuracy)
  validation_loss.append(val_loss_avg/float(j))
  print('\nValidation Accuracy: {}'.format(accuracy))
  if accuracy > best_acc:
    best_acc = accuracy
    best_model = copy.deepcopy(net)

  # Step the scheduler
  pretrained_scheduler.step()
  training_loss.append(loss_avg/float(i))

In [ ]:
from matplotlib import pyplot as plt

fig,ax = plt.subplots(2)
ax[0].plot(validation_accuracy, label = 'Validation Accuracy')
ax[1].plot(training_loss,label='Training Loss')
ax[1].plot(validation_loss,label = 'Validation Loss')
ax[0].grid()
ax[1].grid()
ax[0].legend()
ax[1].legend()

In [0]:
#test phase
net = best_model.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
net.train(False) # Set Network to evaluation mode

running_corrects = 0
for images, labels in tqdm(img_test_dataloader):
  images = images.to(DEVICE)
  labels = labels.to(DEVICE)

  # Forward Pass
  outputs = net(images)

  # Get predictions
  _, preds = torch.max(outputs.data, 1)

  # Update Corrects
  running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(img_test))

print('Test Accuracy: {}'.format(accuracy))

100%|██████████| 6/6 [00:13<00:00,  2.23s/it]

Test Accuracy: 0.844106463878327


**Extra Point**

In [0]:
#Extra point with resnet18
net = resnet18(pretrained=True)
net.fc = nn.Linear(512,NUM_CLASSES)
parameters_to_optimize = net.parameters()
optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

In [0]:
import copy
# By default, everything is loaded to cpu
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

cudnn.benchmark # Calling this optimizes runtime

current_step = 0
best_acc = 0
best_loss = 1000
# Start iterating over the epochs
for epoch in range(NUM_EPOCHS):
  print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_lr()))

  # Iterate over the dataset
  for images, labels in img_train_dataloader:
    # Bring data over the device of choice
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    net.train() # Sets module in training mode

    # PyTorch, by default, accumulates gradients after each backward pass
    # We need to manually set the gradients to zero before starting a new iteration
    optimizer.zero_grad() # Zero-ing the gradients

    # Forward pass to the network
    outputs = net(images)

    # Compute loss based on output and ground truth
    loss = criterion(outputs, labels)

    # Log loss
    if current_step % LOG_FREQUENCY == 0:
      print('Step {}, Loss {}'.format(current_step, loss.item()))

    # Compute gradients for each layer and update weights
    loss.backward()  # backward pass: computes gradients
    optimizer.step() # update weights based on accumulated gradients

    current_step += 1

  net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
  net.train(False) # Set Network to evaluation mode

  running_corrects = 0
  for images, labels in tqdm(img_val_dataloader):
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

      # Forward Pass
    outputs = net(images)

      # Get predictions
    _, preds = torch.max(outputs.data, 1)

      # Update Corrects
    running_corrects += torch.sum(preds == labels.data).data.item()

    # Calculate Accuracy
  accuracy = running_corrects / float(len(img_val))
  print('\nValidation Accuracy: {}'.format(accuracy))
  if accuracy > best_acc:
      best_acc = accuracy
      best_model = copy.deepcopy(net)

  # Step the scheduler
  scheduler.step() 

In [0]:
#test phase
net = best_model.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
net.train(False) # Set Network to evaluation mode

running_corrects = 0
for images, labels in tqdm(img_test_dataloader):
  images = images.to(DEVICE)
  labels = labels.to(DEVICE)

  # Forward Pass
  outputs = net(images)

  # Get predictions
  _, preds = torch.max(outputs.data, 1)

  # Update Corrects
  running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(img_test))

print('Test Accuracy: {}'.format(accuracy))